Day 1 - Clair Agent (100% Local)
First thread using Ollama + Llama 3.2 + LangChain + ChromaDB

In [11]:
# ============================================================
# Cell 1: Setup & Imports
# ============================================================

import os
from datetime import datetime
import time

import arxiv
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from langchain_ollama import OllamaLLM

llmModelCode = "llama3.2:3b"

print("✅ All imports successful")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"🦙 Using: Ollama + {llmModelCode} (100% local)")

✅ All imports successful
📅 Date: 2025-11-18 20:29
🦙 Using: Ollama + llama3.2:3b (100% local)


In [12]:
# ============================================================
# Cell 2: Initialize Ollama LLM
# ============================================================

llm = OllamaLLM(
    model=llmModelCode,
    temperature=0.7,
    max_tokens=512
)

# Test it works
print("🧪 Testing Ollama connection...\n")
test_response = llm.invoke("Say 'Clair is alive' in a calm, technical tone")
print(f"Response: {test_response}\n")
print("✅ Ollama is working!")

🧪 Testing Ollama connection...

Response: Affirmative confirmation: Clair's systems are functioning within normal parameters, indicating that she is currently operational and online.

✅ Ollama is working!


In [13]:
# ============================================================
# Cell 3: Fetch Latest arXiv Paper
# ============================================================

def get_latest_ai_paper():
    """Fetch the most recent AI/ML paper from arXiv"""
    
    print("🔍 Searching arXiv...")
    
    client = arxiv.Client()
    search = arxiv.Search(
        query="cat:cs.AI OR cat:cs.LG OR cat:cs.CL",
        max_results=1,
        sort_by=arxiv.SortCriterion.SubmittedDate
    )

    paper = next(client.results(search))
    
    return {
        'title': paper.title,
        'authors': [a.name for a in paper.authors[:3]],
        'summary': paper.summary[:500],  # Truncate for smaller LLM
        'url': paper.entry_id,
        'published': paper.published.strftime('%Y-%m-%d'),
        'categories': paper.categories
    }

# Fetch paper
paper = get_latest_ai_paper()

print(f"\n📄 Title: {paper['title']}")
print(f"👤 Authors: {', '.join(paper['authors'])}")
print(f"📅 Published: {paper['published']}")
print(f"🔗 URL: {paper['url']}\n")
print(f"📝 Summary: {paper['summary'][:200]}...")

🔍 Searching arXiv...

📄 Title: Scaling Spatial Intelligence with Multimodal Foundation Models
👤 Authors: Zhongang Cai, Ruisi Wang, Chenyang Gu
📅 Published: 2025-11-17
🔗 URL: http://arxiv.org/abs/2511.13719v1

📝 Summary: Despite remarkable progress, multimodal foundation models still exhibit surprising deficiencies in spatial intelligence. In this work, we explore scaling up multimodal foundation models to cultivate s...


In [19]:
# ============================================================
# Cell 4: Generate Thread
# ============================================================

# Optimized prompt for smaller local model
# Note: Simpler instructions work better for 3B models

thread_template = """You are a calm, technical AI researcher explaining papers clearly.

Paper: {title}
Authors: {authors}
Summary: {summary}

Write exactly 3 tweets about this paper. Rules:
- Tweet 1: What problem this solves (under 250 chars)
- Tweet 2: Key technical insight (under 250 chars) 
- Tweet 3: Why it matters (under 250 chars)
- Be clear and technical, not hype
- No buzzwords

Format your response EXACTLY like this:
Tweet 1: [your text]
Tweet 2: [your text]
Tweet 3: [your text]

Now write the 3 tweets:"""

prompt = PromptTemplate(
    input_variables=["title", "authors", "summary"],
    template=thread_template
)

# Build input string from paper data
input_text = prompt.format(
    title=paper['title'],
    authors=", ".join(paper['authors']),
    summary=paper['summary']
)

start_time = time.time()

# Call LLM directly
thread = llm.invoke(input_text)

generation_time = time.time() - start_time

print("✅ Thread generated:\n")
print(thread)

✅ Thread generated:

Here are the three tweets about the paper:

Tweet 1: Multimodal foundation models still struggle with spatial intelligence, leading to subpar performance in tasks like image captioning & visual question answering. This paper addresses this issue by scaling up these models.

Tweet 2: The key technical insight is that the authors use a "principled approach" to scale multimodal models, combining Qwen3-VL & InternVL3 with unified understanding & generation models (e.g., Bagel) to create high-performing & robust spatial intelligence models.

Tweet 3: This research matters because it can improve the accuracy of applications like visual search, image retrieval & object recognition, which rely heavily on spatial intelligence. By scaling up multimodal models, we can unlock better performance in these critical areas.


In [ ]:
# ============================================================
# Cell 5: Save Thread to File
# ============================================================

def save_thread(paper_data, thread_content, gen_time):
    """Save thread as markdown"""
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f"../threads/day01_{timestamp}.md"
    
    # Create threads directory if it doesn't exist
    os.makedirs("../threads", exist_ok=True)
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(f"# Day 1 Thread - Ollama Local\n\n")
        f.write(f"**Paper:** {paper_data['title']}\n")
        f.write(f"**Authors:** {', '.join(paper_data['authors'])}\n")
        f.write(f"**Published:** {paper_data['published']}\n")
        f.write(f"**URL:** {paper_data['url']}\n")
        f.write(f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M')}\n")
        f.write(f"**Generation Time:** {gen_time:.1f}s\n\n")
        f.write("---\n\n")
        f.write(thread_content)
        f.write("\n\n---\n")
        f.write("*Generated by Clair Agent - Day 1*\n")
        f.write("*Stack: Ollama + Llama 3.2 3B + LangChain + ChromaDB*\n")
        f.write("*100% local, $0 API costs*")
    
    return filename

filename = save_thread(paper, thread, generation_time)
print(f"\n💾 Thread saved to: {filename}")


💾 Thread saved to: ../threads/day01_20251118_225254.md


In [ ]:
# ============================================================
# Cell 6: Initialize ChromaDB (For Tomorrow)
# ============================================================

# Setting up vector store for Day 2
# You'll store paper embeddings here

chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    persist_directory="./chroma_db"
))

# Create collection for papers
try:
    collection = chroma_client.create_collection(
        name="arxiv_papers",
        metadata={"description": "AI/ML papers from arXiv"}
    )
    print("\n✅ ChromaDB initialized (empty for now)")
except:
    collection = chroma_client.get_collection("arxiv_papers")
    print("\n✅ ChromaDB collection already exists")

print(f"📊 Current papers in DB: {collection.count()}")


✅ ChromaDB initialized (empty for now)
📊 Current papers in DB: 0


In [25]:
# ============================================================
# Cell 7: Initialize Embedding Model (For Tomorrow)
# ============================================================

# Loading a small, fast embedding model
# You'll use this starting Day 2 for semantic search

print("\n📥 Loading embedding model...")
print("(First time will download ~120MB, takes 30-60 seconds)")

embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Test it
test_embedding = embed_model.encode("This is a test sentence")
print(f"✅ Embedding model loaded (dim: {len(test_embedding)})")


📥 Loading embedding model...
(First time will download ~120MB, takes 30-60 seconds)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedding model loaded (dim: 384)


In [26]:
# ============================================================
# Cell 8: Summary & Next Steps
# ============================================================

print("\n" + "="*60)
print("🎉 DAY 1 COMPLETE - 100% LOCAL STACK")
print("="*60)

print(f"\n✅ Paper fetched: {paper['title'][:50]}...")
print(f"✅ Thread generated in {generation_time:.1f}s")
print(f"✅ Saved to: {filename}")
print(f"✅ ChromaDB initialized")
print(f"✅ Embedding model ready")

print("\n💰 COST:")
print("- Today: $0.00")
print("- Forever: $0.00")
print("- Total API calls: 0")

print("\n📋 TODO NOW:")
print("1. Read the thread in threads/day01_*.md")
print("2. Manually post to X (copy-paste)")
print("3. Post build-in-public update")
print("4. Commit notebook to GitHub")

print("\n🔮 TOMORROW (Day 2):")
print("- Fetch 5 papers instead of 1")
print("- Rank by recency + metadata")
print("- Generate embeddings for each")
print("- Store in ChromaDB")
print("- Semantic search for best paper")

print("\n⚡ MODEL UPGRADE OPTIONS:")
print("- llama3.2:3b → Fast but basic (current)")
print("- llama3.1:8b → Better quality, slower")
print("- qwen2.5:7b → Great for technical content")
print(f"\nTo switch: ollama pull [model]")

print(f"\n⏱️  Total time: ~{(generation_time + 30):.0f} minutes")
print("💪 100% local. 100% free. 100% yours.")


🎉 DAY 1 COMPLETE - 100% LOCAL STACK

✅ Paper fetched: Scaling Spatial Intelligence with Multimodal Found...
✅ Thread generated in 49.7s
✅ Saved to: ../threads/day01_20251118_225254.md
✅ ChromaDB initialized
✅ Embedding model ready

💰 COST:
- Today: $0.00
- Forever: $0.00
- Total API calls: 0

📋 TODO NOW:
1. Read the thread in threads/day01_*.md
2. Manually post to X (copy-paste)
3. Post build-in-public update
4. Commit notebook to GitHub

🔮 TOMORROW (Day 2):
- Fetch 5 papers instead of 1
- Rank by recency + metadata
- Generate embeddings for each
- Store in ChromaDB
- Semantic search for best paper

⚡ MODEL UPGRADE OPTIONS:
- llama3.2:3b → Fast but basic (current)
- llama3.1:8b → Better quality, slower
- qwen2.5:7b → Great for technical content

To switch: ollama pull [model]

⏱️  Total time: ~80 minutes
💪 100% local. 100% free. 100% yours.
